## DATA623 Assignment 4
#### Jordan Swanson 10005366

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportions_ztest

## Introduction
The first step in approaching this question was to retrieve the original data and the paper published from it [1] and using the source to answer one of the provided questions for the assignment. The question I've chosen to answer is **Is there a difference in the outcome (i.e. acute ischemic stroke) between the aspirin and high heparin groups?** Right away we can import the data data into pandas and check to see how much data we're working with:

In [2]:
data = pd.read_csv('IST_corrected.csv', encoding='latin', low_memory=False)
data.shape

(19435, 112)

## Methods
We have 19,435 rows, which is great, as that's the exact same number that's mentioned in the paper title. We also have 112 columns of data, but to answer our question we only need a few, so we'll focus on those instead and use the `IST_variables.csv` file to guide us as to which variables to use. We'll want whatever demographic data we have the subjects including knowing if they were hospitalized due to ischemic stroke, as well as the codings to know which group they were randomized to. Finally, we'll need information on the target outcome, which for simplicity for this analysis we'll consider death at either 14 days or 6 months:

SEX: M=male; F=female <br>
AGE: Age in years <br>
DDIAGISC: Ischaemic stroke <br>
RXASP: Trial aspirin allocated (Y/N) <br>
RXHEP: Trial heparin allocated (M/L/N) [M is coded as H=high in pilot] <br>
ID14: Indicator of death at 14 days <br>
OCCODE: Six month outcome (1-dead/2-dependent/3-not recovered/4-recovered/8 or 9 – missing status <br>

In [3]:
data = data[data['DDIAGISC'] == 'Y'][['SEX', 'AGE', 'RXASP', 'RXHEP', 'ID14', 'OCCODE']]

However, for us to really make a fair judgement of the two groups we're looking at, we need to acknowledge how the study was designed. As a factorial study design, patients could be in one of two levels of treatment with aspirin (Yes or No), and simultaneously be in one of three treatment levels for heparin (No, Low, High). That's a total of six possible treatment groups, and to isolate the effect of the treatments in question, we'll reduce our dataset to just two: aspirin with **no** heparin, and high heparin with **no** aspirin, which we'll also denote as Group A and Group B, respectively.

To filter our data, we'll have to include only patients who meet *both* of the above criteria for each group, as well as ensure that we have data about the patient at the 6 month mark (if applicable):

In [4]:
dataA = data[(data['RXASP'] == 'Y') & (data['RXHEP'] == 'N') & (~data['OCCODE'].isin([8,9]))]
dataB = data[(data['RXHEP'].isin(['H', 'M'])) & (data['RXASP'] == 'N') & (~data['OCCODE'].isin([8,9]))]

## Results
We can quickly see that the two groups are remarkably similar in the distribution of the ages of the patients, though Group B contains only around half as many patients as Group A, as well as a slightly higher proportion of male patients:

In [5]:
print('Group A age descriptives')
display(dataA['AGE'].describe())
print('\nGroup A sex breakdown')
display(dataA['SEX'].value_counts(normalize=True))
print('\nGroup B age descriptives')
display(dataB['AGE'].describe())
print('\nGroup B sex breakdown')
display(dataB['SEX'].value_counts(normalize=True))

Group A age descriptives


count    4316.000000
mean       71.506024
std        11.580829
min        25.000000
25%        65.000000
50%        73.000000
75%        80.000000
max        98.000000
Name: AGE, dtype: float64


Group A sex breakdown


M    0.529425
F    0.470575
Name: SEX, dtype: float64


Group B age descriptives


count    2181.000000
mean       71.516277
std        11.293025
min        16.000000
25%        65.000000
50%        73.000000
75%        80.000000
max        98.000000
Name: AGE, dtype: float64


Group B sex breakdown


M    0.552499
F    0.447501
Name: SEX, dtype: float64

## Analysis

Our first step will be to compare the 14-day survival between the two groups. As this is coded in our data as a binary variable, with 1 being death and 0 being alive, we can compare the proportion of death between the two groups with a two-sample *Z* test at $\alpha=0.05$:

*$H_0$*: The two groups have similar proportions of death at 14 days<br>
*$H_a$*: The two groups differ in their proportion of death at 14 days


In [6]:
deathA14, sizeA14 = dataA['ID14'].sum(), len(dataA['ID14'])
deathB14, sizeB14 = dataB['ID14'].sum(), len(dataB['ID14'])
deaths14 = np.array([deathA14, deathB14])
samples14 = np.array([sizeA14, sizeB14])
stat14, pval14 = proportions_ztest(deaths14, samples14, alternative='two-sided')

print(f'The Z-test of different proportions has a p-value of {round(pval14, 4)}')

The Z-test of different proportions has a p-value of 0.9252


As we can see from above, the two groups are extremely similar in the proportion of death at 14 days, and no discernible difference can be found. 
<br>
The secondary outcome we're interested in is the incidence of death at 6 months following initial hospitalization. To do that, we'll use the same statistical test for the 6-month follow-up ($\alpha=0.05$):

*$H_0$*: The two groups have similar proportions of death by the 6 month follow-up<br>
*$H_a$*: The two groups differ in their proportion of death at the 6 month follow-up

In [7]:
# The data for 6-months is coded such that while there's only one value for death, there are three other outcomes that are not death
# We've already removed values for missing data, so all non-death values can be lumped together as such:
deathA6, sizeA6 = dataA[dataA['OCCODE']==1]['OCCODE'].count(), len(dataA['ID14'])
deathB6, sizeB6 = dataB[dataB['OCCODE']!=1]['OCCODE'].count(), len(dataB['ID14'])
deaths6 = np.array([deathA6, deathB6])
samples6 = np.array([sizeA6, sizeB6])
stat6, pval6 = proportions_ztest(deaths6, samples6, alternative='two-sided')

print(f'The Z-test of different proportions has a p-value of {pval6}')

The Z-test of different proportions has a p-value of 0.0


As seen above, the *p*-value is vanishingly small (<0.00001), so we have sufficient evidence to reject the null hypothesis of no difference, and accept that there's a significant difference between the groups. The proportions at 6 months:

In [10]:
print(f'Group A proportion of death by 6 months: {round(deathA6 / sizeA6, 4)}')
print(f'Group B proportion of death by 6 months: {round(deathB6 / sizeB6, 4)}')

Group A proportion of death by 6 months: 0.2023
Group B proportion of death by 6 months: 0.7822


## Conclusion

With the significant two-sample proportional *Z*-test, we can conclude that the two groups have a significant difference in the proportion of death at 6 months. Though there's no detectable difference at the 14-day mark, it would appear that **ischemic stroke patients who are treated with aspirin alone (Group A) have a higher survivability rate at 6 months than those who undergo treatment with high doses of heparin (Group B).** Importantly, though the original study had more groupings for the participants, they too arrived at the same conclusion [1].

### References
1. International Stroke Trial Collaborative Group. (1997). The International Stroke Trial (IST): a randomised trial of aspirin, subcutaneous heparin, both, or neither among 19 435 patients with acute ischaemic stroke. *The Lancet* 349, 1569-1581. https://doi.org/10.1016/S0140-6736(97)04011-7